In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


In [8]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/신규_기존.csv',)
df.head()

,cust_no,2014_1,2014_2,2015_1,2015_2,2014_1_신규,2014_2_신규,2015_1_신규,2015_2_이탈
0,1,19925355.0,34116312.0,17898365.0,13102518.0,기존,기존,기존,기존
1,2,24323230.0,21922100.0,27791610.0,25843820.0,기존,기존,기존,기존
2,3,851215.0,1042554.0,592280.0,679118.0,기존,기존,기존,기존
3,4,5042106.0,4257510.0,3656937.0,3737257.0,기존,기존,기존,기존
4,5,5460150.0,2225730.0,2934930.0,2616260.0,기존,기존,기존,기존


In [9]:
# 각 시점에 구매액이 있으면 기존 고객으로 분류
def cust(x1, x2, x3, x4):
    if (x1 == '기존')&(x2 == '기존')&(x3 == '기존')&(x4 == '기존'):
        return '기존'
    else:
        return '신규'
        
df['cust'] = df.apply(lambda x : cust(x['2014_1_신규'],x['2014_2_신규'],x['2015_1_신규'],x['2015_2_이탈']),axis=1)

# 기존고객만 분류한 데이터프레임 df 지정, 수치데이터만 남기고 나머지는 삭제
df = df[df['cust'] == '기존']
df = df.iloc[:,:-5]

# 분석 대상 target : 2015년 하반기, 2014년 상반기 구매금액 차이
# target_id : 구매금액 증감 여부 
df['target'] = df['2015_2'] - df['2014_1']

# 구매 증감 사용자 함수
def di(x):
    if x > 0 : return '증가'
    elif x == 0 : return '유지'
    else: return '감소'

# 구매 증감 여부 컬럼 생성
df['target_di'] = df['target'].apply(di)

# 등급 구하기
def cust_level(x):
    if x >= 34350000: return 'VIP'
    elif x >= 20150000: return '최우수'
    elif x >= 8700000: return '우수'
    elif x == 0: return '없음'
    else: return '일반'

# 각 대상 시기별 등급 컬럼 생성
df['14_1_level'] = df['2014_1'].apply(cust_level)
df['15_2_level'] = df['2015_2'].apply(cust_level)

# 2014년 상 - 2015년 하반기 등급 변화 컬럼 생성 
df['level_ch'] = df[['14_1_level', '15_2_level']].apply('_'.join, axis = 1)

df

,cust_no,2014_1,2014_2,2015_1,2015_2,target,target_di,14_1_level,15_2_level,level_ch
0,1,19925355.0,34116312.0,17898365.0,13102518.0,-6822837.0,감소,우수,우수,우수_우수
1,2,24323230.0,21922100.0,27791610.0,25843820.0,1520590.0,증가,최우수,최우수,최우수_최우수
2,3,851215.0,1042554.0,592280.0,679118.0,-172097.0,감소,일반,일반,일반_일반
3,4,5042106.0,4257510.0,3656937.0,3737257.0,-1304849.0,감소,일반,일반,일반_일반
4,5,5460150.0,2225730.0,2934930.0,2616260.0,-2843890.0,감소,일반,일반,일반_일반
...,...,...,...,...,...,...,...,...,...,...
19355,19356,170160.0,1536090.0,2104968.0,2434587.0,2264427.0,증가,일반,일반,일반_일반
19356,19357,91950.0,3281346.0,3044944.0,2246675.0,2154725.0,증가,일반,일반,일반_일반
19358,19359,43190.0,1286327.0,837240.0,1092380.0,1049190.0,증가,일반,일반,일반_일반
19359,19360,368050.0,2060040.0,1412140.0,886060.0,518010.0,증가,일반,일반,일반_일반


In [4]:
import pandas as pd
purprd = pd.read_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/purprd.csv')
purprd.head(3) 

,channel,receipt_no,code_dae,code_jung,code_so,cust_no,branch_no,pur_date,pur_hour,pur_amount,cat_dae,cat_jung,cat_so,year,month,yoil,half,week,pur_time,period,pur_week_time
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420,가공식품,탄산음료,사이다,2014,2,토,1,주말,밤,2014_1,주말_밤
1,B,8664007,15,1504,B150401,11303,44,2014-02-22,13,2400,가공식품,탄산음료,사이다,2014,2,토,1,주말,오후,2014_1,주말_오후
2,B,8919520,15,1504,B150401,13210,48,2015-09-21,19,5250,가공식품,탄산음료,사이다,2015,9,월,2,주중,밤,2015_2,주중_밤


In [10]:
# 분석 대상 품목별 구매금액 데이터 프레임 생성하기
purprd14_1 = purprd[purprd['period'] == '2014_1']
purprd15_2 = purprd[purprd['period'] == '2015_2']
purprd14_1.shape, purprd15_2.shape

((6709382, 21), (7247418, 21))

In [11]:
purprd['channel_item'] = purprd[['channel', 'cat_dae']].apply('_'.join, axis = 1)
pd.pivot_table(purprd, index = 'cust_no', columns = 'channel_item', values = 'pur_amount', aggfunc = 'sum')

channel_item,A_가공식품,A_가구/인테리어,A_가전,A_교육/문화용품,A_기타,A_명품,A_신선식품,A_외식,A_의류,A_일상용품,A_전문스포츠/레저,A_패션잡화,B_가공식품,B_가구/인테리어,B_가전,B_교육/문화용품,B_기타,B_명품,B_신선식품,B_외식,B_의류,B_의약품,B_일상용품,B_전문스포츠/레저,B_패션잡화,C_가공식품,C_가구/인테리어,C_가전,C_교육/문화용품,C_신선식품,C_의류,C_의약품,C_일상용품,C_전문스포츠/레저,C_패션잡화,D_가공식품,D_가전,D_교육/문화용품,D_의약품,D_일상용품,D_전문스포츠/레저,D_패션잡화
cust_no,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5886110.0,5662890.0,11153100.0,NaN,NaN,503500.0,2692730.0,1848970.0,39917930.0,8599000.0,3127830.0,3835090.0,144440.0,NaN,NaN,NaN,NaN,NaN,184456.0,294100.0,NaN,8000.0,159980.0,NaN,NaN,261790.0,NaN,NaN,NaN,698324.0,NaN,21800.0,42510.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4707110.0,938730.0,7372350.0,NaN,NaN,1978060.0,8871350.0,1383400.0,28507040.0,33005640.0,10782560.0,1908610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32700.0,167950.0,NaN,8800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,214460.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80000.0,NaN,NaN,NaN,1960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,NaN,NaN,1993380.0,NaN,26900.0,NaN,805687.0,NaN,NaN,244440.0,NaN,7800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1826170.0,463000.0,NaN,NaN,NaN,NaN,1166440.0,1418470.0,6222600.0,1373360.0,1215450.0,589120.0,349893.0,37800.0,NaN,NaN,NaN,NaN,279153.0,NaN,531700.0,9800.0,131620.0,NaN,254100.0,148910.0,NaN,NaN,NaN,570544.0,47700.0,30480.0,16080.0,3520.0,7900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5655770.0,212300.0,49900.0,NaN,NaN,57000.0,1480680.0,1213840.0,1888350.0,2264530.0,134280.0,217570.0,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44000.0,NaN,NaN,2850.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19379,9700.0,NaN,NaN,NaN,NaN,55800.0,NaN,145050.0,816780.0,41640.0,NaN,941050.0,25280.0,NaN,NaN,NaN,35000.0,NaN,NaN,NaN,60800.0,4000.0,175900.0,NaN,NaN,21420.0,NaN,NaN,NaN,32470.0,NaN,NaN,680.0,NaN,NaN,408870.0,9900.0,NaN,78350.0,2846440.0,21000.0,49150.0
19380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18300.0,NaN,34000.0,NaN,NaN,7100.0,NaN,NaN,NaN,NaN,NaN,24180.0,NaN,NaN,NaN,NaN,NaN,29000.0,77540.0,NaN,NaN,NaN,51887.0,NaN,NaN,17410.0,NaN,NaN,475740.0,133440.0,3000.0,24980.0,3016400.0,NaN,NaN
19381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.0,350300.0,47500.0,NaN,NaN,2112583.0,109700.0,NaN,2500.0,NaN,NaN,2370306.0,146800.0,150700.0,35800.0,581390.0,36200.0,68600.0,20990.0,NaN,NaN,NaN,NaN,NaN,NaN,2500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
channel_by_item_df = pd.pivot_table(purprd, index = 'cust_no', columns = 'channel_item', values = 'pur_amount', aggfunc = 'sum')

In [13]:
column_name = channel_by_item_df.columns.tolist()

In [16]:
column_name[:12]

['A_가공식품',
 'A_가구/인테리어',
 'A_가전',
 'A_교육/문화용품',
 'A_기타',
 'A_명품',
 'A_신선식품',
 'A_외식',
 'A_의류',
 'A_일상용품',
 'A_전문스포츠/레저',
 'A_패션잡화']

In [22]:
column_name[12:25]

['B_가공식품',
 'B_가구/인테리어',
 'B_가전',
 'B_교육/문화용품',
 'B_기타',
 'B_명품',
 'B_신선식품',
 'B_외식',
 'B_의류',
 'B_의약품',
 'B_일상용품',
 'B_전문스포츠/레저',
 'B_패션잡화']

In [24]:
column_name[25:35]

['C_가공식품',
 'C_가구/인테리어',
 'C_가전',
 'C_교육/문화용품',
 'C_신선식품',
 'C_의류',
 'C_의약품',
 'C_일상용품',
 'C_전문스포츠/레저',
 'C_패션잡화']

In [30]:
column_name[35:]

['D_가공식품', 'D_가전', 'D_교육/문화용품', 'D_의약품', 'D_일상용품', 'D_전문스포츠/레저', 'D_패션잡화']

In [27]:
column_name

['A_가공식품',
 'A_가구/인테리어',
 'A_가전',
 'A_교육/문화용품',
 'A_기타',
 'A_명품',
 'A_신선식품',
 'A_외식',
 'A_의류',
 'A_일상용품',
 'A_전문스포츠/레저',
 'A_패션잡화',
 'B_가공식품',
 'B_가구/인테리어',
 'B_가전',
 'B_교육/문화용품',
 'B_기타',
 'B_명품',
 'B_신선식품',
 'B_외식',
 'B_의류',
 'B_의약품',
 'B_일상용품',
 'B_전문스포츠/레저',
 'B_패션잡화',
 'C_가공식품',
 'C_가구/인테리어',
 'C_가전',
 'C_교육/문화용품',
 'C_신선식품',
 'C_의류',
 'C_의약품',
 'C_일상용품',
 'C_전문스포츠/레저',
 'C_패션잡화',
 'D_가공식품',
 'D_가전',
 'D_교육/문화용품',
 'D_의약품',
 'D_일상용품',
 'D_전문스포츠/레저',
 'D_패션잡화']

In [31]:
# 해당 시기의 각 고객의 각 제휴사별 품목, 구매비중을 구하는 사용자 함수
def channel_by_item(purprd):
  purprd['channel_item'] = purprd[['channel', 'cat_dae']].apply('_'.join, axis = 1)
  channel_by_item_df = pd.pivot_table(purprd, index = 'cust_no', columns = 'channel_item', values = 'pur_amount', aggfunc = 'sum')
  # 컬럼명
  column_name = channel_by_item_df.columns.tolist()
  # 총 구매금액 (전체 제휴사 기준)
  channel_by_item_df['sum_amt'] = channel_by_item_df[column_name].sum(axis = 1)

  # A 제휴사 최대 구매품목(idxmax)
  channel_by_item_df['max_item_A'] = channel_by_item_df[column_name[:12]].idxmax(axis = 1)
  # A 제휴사 최대 구매금액(max)
  channel_by_item_df['max_item_amt_A'] = channel_by_item_df[column_name[:12]].max(axis = 1)
  # A 제휴사 최대 구매품목의 구매비중
  channel_by_item_df['max_item_ratio_A'] = channel_by_item_df[column_name[:12]].max(axis = 1) / channel_by_item_df['sum_amt']


  # B 제휴사 최대 구매품목(idxmax)
  channel_by_item_df['max_item_B'] = channel_by_item_df[column_name[12:25]].idxmax(axis = 1)
  # B 제휴사 최대 구매금액(max)
  channel_by_item_df['max_item_amt_B'] = channel_by_item_df[column_name[12:25]].max(axis = 1)
  # b 제휴사 최대 구매품목의 구매비중
  channel_by_item_df['max_item_ratio_B'] = channel_by_item_df[column_name[12:25]].max(axis = 1) / channel_by_item_df['sum_amt']


  # C 제휴사 최대 구매품목(idxmax)
  channel_by_item_df['max_item_C'] = channel_by_item_df[column_name[25:35]].idxmax(axis = 1)
  # C 제휴사 최대 구매금액(max)
  channel_by_item_df['max_item_amt_C'] = channel_by_item_df[column_name[25:35]].max(axis = 1)
  # C 제휴사 최대 구매품목의 구매비중
  channel_by_item_df['max_item_ratio_C'] = channel_by_item_df[column_name[25:35]].max(axis = 1) / channel_by_item_df['sum_amt']


  # D 제휴사 최대 구매품목(idxmax)
  channel_by_item_df['max_item_D'] = channel_by_item_df[column_name[35:]].idxmax(axis = 1)
  # D 제휴사 최대 구매금액(max)
  channel_by_item_df['max_item_amt_D'] = channel_by_item_df[column_name[35:]].max(axis = 1)
  # D 제휴사 최대 구매품목의 구매비중
  channel_by_item_df['max_item_ratio_D'] = channel_by_item_df[column_name[35:]].max(axis = 1) / channel_by_item_df['sum_amt']


  return channel_by_item_df

def channel_by_item_comparison(df, purprd1, purprd2):
  # 1번 데이터프레임
  channel_by_item1 = channel_by_item(purprd1)
  column_name1 = channel_by_item1.columns.tolist()
  # 1번 데이터프레임 컬럼명 변경
  channel_by_item1.columns = [x + '1' for x in column_name1]
  channel_by_item1.reset_index(inplace = True)

  # 2번 데이터프레임
  channel_by_item2 = channel_by_item(purprd2)
  column_name2 = channel_by_item2.columns.tolist()
  
  # 2번 데이터프레임 컬럼명 변경
  channel_by_item2.columns = [x + '2' for x in column_name2]
  channel_by_item2.reset_index(inplace = True)

  # 1,2번 데이터프레임 merge
  channel_by_item_df = pd.merge(channel_by_item1, channel_by_item2, on = 'cust_no')
  channel_by_item_df = pd.merge(df, channel_by_item_df, on = 'cust_no')

  return channel_by_item_df

comparison_by_item_channel = channel_by_item_comparison(df, purprd14_1, purprd15_2)
comparison_by_item_channel

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,cust_no,2014_1,2014_2,2015_1,2015_2,target,target_di,14_1_level,15_2_level,level_ch,A_가공식품1,A_가구/인테리어1,A_가전1,A_교육/문화용품1,A_기타1,A_명품1,A_신선식품1,A_외식1,A_의류1,A_일상용품1,A_전문스포츠/레저1,A_패션잡화1,B_가공식품1,B_가구/인테리어1,B_가전1,B_교육/문화용품1,B_기타1,B_명품1,B_신선식품1,B_외식1,B_의류1,B_의약품1,B_일상용품1,B_전문스포츠/레저1,B_패션잡화1,C_가공식품1,C_가구/인테리어1,C_가전1,C_교육/문화용품1,C_신선식품1,...,B_교육/문화용품2,B_기타2,B_명품2,B_신선식품2,B_외식2,B_의류2,B_의약품2,B_일상용품2,B_전문스포츠/레저2,B_패션잡화2,C_가공식품2,C_가구/인테리어2,C_가전2,C_교육/문화용품2,C_신선식품2,C_의류2,C_의약품2,C_일상용품2,C_전문스포츠/레저2,C_패션잡화2,D_가공식품2,D_가전2,D_교육/문화용품2,D_의약품2,D_일상용품2,D_전문스포츠/레저2,D_패션잡화2,sum_amt2,max_item_A2,max_item_amt_A2,max_item_ratio_A2,max_item_B2,max_item_amt_B2,max_item_ratio_B2,max_item_C2,max_item_amt_C2,max_item_ratio_C2,max_item_D2,max_item_amt_D2,max_item_ratio_D2
0,1,19925355.0,34116312.0,17898365.0,13102518.0,-6822837.0,감소,우수,우수,우수_우수,3492840.0,177650.0,NaN,NaN,NaN,NaN,787820.0,266950.0,9612380.0,2208520.0,2143200.0,1013850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21800.0,NaN,NaN,NaN,NaN,NaN,20990.0,NaN,NaN,NaN,178605.0,...,NaN,NaN,NaN,NaN,184900.0,NaN,NaN,NaN,NaN,NaN,29230.0,NaN,NaN,NaN,71938.0,NaN,NaN,15560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13102518.0,A_의류,7788970.0,0.594464,B_외식,184900.0,0.014112,C_신선식품,71938.0,0.005490,NaN,NaN,NaN
1,2,24323230.0,21922100.0,27791610.0,25843820.0,1520590.0,증가,최우수,최우수,최우수_최우수,1527630.0,530700.0,1421700.0,NaN,NaN,NaN,3353800.0,472600.0,3577890.0,8661920.0,3949760.0,796730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,42000.0,NaN,NaN,25843820.0,A_의류,9904500.0,0.383244,NaN,NaN,NaN,NaN,NaN,NaN,D_일상용품,42000.0,0.001625
2,3,851215.0,1042554.0,592280.0,679118.0,-172097.0,감소,일반,일반,일반_일반,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,616100.0,NaN,NaN,NaN,132165.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,351740.0,NaN,NaN,NaN,257028.0,NaN,NaN,62550.0,NaN,7800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,679118.0,NaN,NaN,NaN,NaN,NaN,NaN,C_가공식품,351740.0,0.517936,NaN,NaN,NaN
3,4,5042106.0,4257510.0,3656937.0,3737257.0,-1304849.0,감소,일반,일반,일반_일반,659890.0,NaN,NaN,NaN,NaN,NaN,367480.0,246610.0,2272650.0,159010.0,402100.0,520520.0,27800.0,5200.0,NaN,NaN,NaN,NaN,25680.0,NaN,15000.0,NaN,6380.0,NaN,NaN,44060.0,NaN,NaN,NaN,242366.0,...,NaN,NaN,NaN,69773.0,NaN,307500.0,NaN,79380.0,NaN,184700.0,17520.0,NaN,NaN,NaN,154905.0,19800.0,10160.0,NaN,3520.0,7900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3737257.0,A_의류,1289780.0,0.345114,B_의류,307500.0,0.082280,C_신선식품,154905.0,0.041449,NaN,NaN,NaN
4,5,5460150.0,2225730.0,2934930.0,2616260.0,-2843890.0,감소,일반,일반,일반_일반,2016000.0,NaN,49900.0,NaN,NaN,57000.0,789820.0,474240.0,542900.0,1312640.0,63080.0,138570.0,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29300.0,NaN,NaN,NaN,NaN,NaN,NaN,2616260.0,A_가공식품,1235890.0,0.472388,NaN,NaN,NaN,NaN,NaN,NaN,D_가공식품,29300.0,0.011199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19142,19356,170160.0,1536090.0,2104968.0,2434587.0,2264427.0,증가,일반,일반,일반_일반,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87670.0,NaN,NaN,NaN,78500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1267480.0,NaN,NaN,NaN,614357.0,NaN,26760.0,313090.0,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2434587.0,A_의류,140400.0,0.057669,NaN,NaN,NaN,C_가공식품,1267480.0,0.520614,NaN,NaN,NaN
19143,19357,91950.0,3281346.0,3044944.0,2246675.0,2154725.0,증가,일반,일반,일반_일반,NaN,NaN,NaN,NaN,NaN,NaN,9700.0,NaN,43000.0,NaN,NaN,NaN,29650.0,NaN,NaN,NaN,NaN,NaN,9600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,108370.0,NaN,NaN,397037

In [33]:
comparison_by_item_channel.head(3)

,cust_no,2014_1,2014_2,2015_1,2015_2,target,target_di,14_1_level,15_2_level,level_ch,A_가공식품1,A_가구/인테리어1,A_가전1,A_교육/문화용품1,A_기타1,A_명품1,A_신선식품1,A_외식1,A_의류1,A_일상용품1,A_전문스포츠/레저1,A_패션잡화1,B_가공식품1,B_가구/인테리어1,B_가전1,B_교육/문화용품1,B_기타1,B_명품1,B_신선식품1,B_외식1,B_의류1,B_의약품1,B_일상용품1,B_전문스포츠/레저1,B_패션잡화1,C_가공식품1,C_가구/인테리어1,C_가전1,C_교육/문화용품1,C_신선식품1,...,B_교육/문화용품2,B_기타2,B_명품2,B_신선식품2,B_외식2,B_의류2,B_의약품2,B_일상용품2,B_전문스포츠/레저2,B_패션잡화2,C_가공식품2,C_가구/인테리어2,C_가전2,C_교육/문화용품2,C_신선식품2,C_의류2,C_의약품2,C_일상용품2,C_전문스포츠/레저2,C_패션잡화2,D_가공식품2,D_가전2,D_교육/문화용품2,D_의약품2,D_일상용품2,D_전문스포츠/레저2,D_패션잡화2,sum_amt2,max_item_A2,max_item_amt_A2,max_item_ratio_A2,max_item_B2,max_item_amt_B2,max_item_ratio_B2,max_item_C2,max_item_amt_C2,max_item_ratio_C2,max_item_D2,max_item_amt_D2,max_item_ratio_D2
0,1,19925355.0,34116312.0,17898365.0,13102518.0,-6822837.0,감소,우수,우수,우수_우수,3492840.0,177650.0,NaN,NaN,NaN,NaN,787820.0,266950.0,9612380.0,2208520.0,2143200.0,1013850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21800.0,NaN,NaN,NaN,NaN,NaN,20990.0,NaN,NaN,NaN,178605.0,...,NaN,NaN,NaN,NaN,184900.0,NaN,NaN,NaN,NaN,NaN,29230.0,NaN,NaN,NaN,71938.0,NaN,NaN,15560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13102518.0,A_의류,7788970.0,0.594464,B_외식,184900.0,0.014112,C_신선식품,71938.0,0.005490,NaN,NaN,NaN
1,2,24323230.0,21922100.0,27791610.0,25843820.0,1520590.0,증가,최우수,최우수,최우수_최우수,1527630.0,530700.0,1421700.0,NaN,NaN,NaN,3353800.0,472600.0,3577890.0,8661920.0,3949760.0,796730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,42000.0,NaN,NaN,25843820.0,A_의류,9904500.0,0.383244,NaN,NaN,NaN,NaN,NaN,NaN,D_일상용품,42000.0,0.001625
2,3,851215.0,1042554.0,592280.0,679118.0,-172097.0,감소,일반,일반,일반_일반,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,616100.0,NaN,NaN,NaN,132165.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,351740.0,NaN,NaN,NaN,257028.0,NaN,NaN,62550.0,NaN,7800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,679118.0,NaN,NaN,NaN,NaN,NaN,NaN,C_가공식품,351740.0,0.517936,NaN,NaN,NaN


In [69]:
# 관심있는 컬럼만 선택
cdf = comparison_by_item_channel[['cust_no', 'level_ch', 'target_di',
                                  'max_item_A1', 'max_item_ratio_A1', 'max_item_B1', 'max_item_ratio_B1', 'max_item_C1', 'max_item_ratio_C1', 'max_item_D1', 'max_item_ratio_D1',
                                  'max_item_A2', 'max_item_ratio_A2', 'max_item_B2', 'max_item_ratio_B2', 'max_item_C2', 'max_item_ratio_C2', 'max_item_D2', 'max_item_ratio_D2']]
cdf

,cust_no,level_ch,target_di,max_item_A1,max_item_ratio_A1,max_item_B1,max_item_ratio_B1,max_item_C1,max_item_ratio_C1,max_item_D1,max_item_ratio_D1,max_item_A2,max_item_ratio_A2,max_item_B2,max_item_ratio_B2,max_item_C2,max_item_ratio_C2,max_item_D2,max_item_ratio_D2
0,1,우수_우수,감소,A_의류,0.482420,B_외식,0.001094,C_신선식품,0.008964,NaN,NaN,A_의류,0.594464,B_외식,0.014112,C_신선식품,0.005490,NaN,NaN
1,2,최우수_최우수,증가,A_일상용품,0.356117,B_일상용품,0.001254,NaN,NaN,NaN,NaN,A_의류,0.383244,NaN,NaN,NaN,NaN,D_일상용품,0.001625
2,3,일반_일반,감소,A_의류,0.093983,NaN,NaN,C_가공식품,0.723789,NaN,NaN,NaN,NaN,NaN,NaN,C_가공식품,0.517936,NaN,NaN
3,4,일반_일반,감소,A_의류,0.450734,B_가공식품,0.005514,C_신선식품,0.048068,NaN,NaN,A_의류,0.345114,B_의류,0.082280,C_신선식품,0.041449,NaN,NaN
4,5,일반_일반,감소,A_가공식품,0.369221,B_신선식품,0.002930,NaN,NaN,NaN,NaN,A_가공식품,0.472388,NaN,NaN,NaN,NaN,D_가공식품,0.011199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19142,19356,일반_일반,증가,NaN,NaN,NaN,NaN,C_가공식품,0.515221,NaN,NaN,A_의류,0.057669,NaN,NaN,C_가공식품,0.520614,NaN,NaN
19143,19357,일반_일반,증가,A_의류,0.467645,B_가공식품,0.322458,NaN,NaN,NaN,NaN,A_의류,0.135756,B_가공식품,0.447603,NaN,NaN,NaN,NaN
19144,19359,일반_일반,증가,NaN,NaN,NaN,NaN,C_신선식품,0.575596,NaN,NaN,NaN,NaN,NaN,NaN,C_가공식품,0.728336,NaN,NaN
19145,19360,일반_일반,증가,NaN,NaN,NaN,NaN,NaN,NaN,D_일상용품,0.727347,A_패션잡화,0.242647,NaN,NaN,NaN,NaN,D_일상용품,0.365720


In [71]:
# 2기로 잘못쓴거 4기로 컬럼명 변경
cdf.columns = ['cust_no', 'level_ch', 'target_di',
              'max_item_A1', 'max_item_ratio_A1', 'max_item_B1', 'max_item_ratio_B1', 'max_item_C1', 'max_item_ratio_C1', 'max_item_D1', 'max_item_ratio_D1',
              'max_item_A4', 'max_item_ratio_A4', 'max_item_B4', 'max_item_ratio_B4', 'max_item_C4', 'max_item_ratio_C4', 'max_item_D4', 'max_item_ratio_D4']
cdf.head()

,cust_no,level_ch,target_di,max_item_A1,max_item_ratio_A1,max_item_B1,max_item_ratio_B1,max_item_C1,max_item_ratio_C1,max_item_D1,max_item_ratio_D1,max_item_A4,max_item_ratio_A4,max_item_B4,max_item_ratio_B4,max_item_C4,max_item_ratio_C4,max_item_D4,max_item_ratio_D4
0,1,우수_우수,감소,A_의류,0.482420,B_외식,0.001094,C_신선식품,0.008964,NaN,NaN,A_의류,0.594464,B_외식,0.014112,C_신선식품,0.005490,NaN,NaN
1,2,최우수_최우수,증가,A_일상용품,0.356117,B_일상용품,0.001254,NaN,NaN,NaN,NaN,A_의류,0.383244,NaN,NaN,NaN,NaN,D_일상용품,0.001625
2,3,일반_일반,감소,A_의류,0.093983,NaN,NaN,C_가공식품,0.723789,NaN,NaN,NaN,NaN,NaN,NaN,C_가공식품,0.517936,NaN,NaN
3,4,일반_일반,감소,A_의류,0.450734,B_가공식품,0.005514,C_신선식품,0.048068,NaN,NaN,A_의류,0.345114,B_의류,0.082280,C_신선식품,0.041449,NaN,NaN
4,5,일반_일반,감소,A_가공식품,0.369221,B_신선식품,0.002930,NaN,NaN,NaN,NaN,A_가공식품,0.472388,NaN,NaN,NaN,NaN,D_가공식품,0.011199


In [72]:
# A_, B_, C_, D_, 데이터값에서 삭제

import warnings
warnings.filterwarnings('ignore')

cdf['max_item_A1'] = cdf['max_item_A1'].astype(str).apply(lambda x : re.sub('A_', '', x))
cdf['max_item_A4'] = cdf['max_item_A4'].astype(str).apply(lambda x : re.sub('A_', '', x))

cdf['max_item_B1'] = cdf['max_item_B1'].astype(str).apply(lambda x : re.sub('B_', '', x))
cdf['max_item_B4'] = cdf['max_item_B4'].astype(str).apply(lambda x : re.sub('B_', '', x))

cdf['max_item_C1'] = cdf['max_item_C1'].astype(str).apply(lambda x : re.sub('C_', '', x))
cdf['max_item_C4'] = cdf['max_item_C4'].astype(str).apply(lambda x : re.sub('C_', '', x))

cdf['max_item_D1'] = cdf['max_item_D1'].astype(str).apply(lambda x : re.sub('D_', '', x))
cdf['max_item_D4'] = cdf['max_item_D4'].astype(str).apply(lambda x : re.sub('D_', '', x))

cdf.head(3)

,cust_no,level_ch,target_di,max_item_A1,max_item_ratio_A1,max_item_B1,max_item_ratio_B1,max_item_C1,max_item_ratio_C1,max_item_D1,max_item_ratio_D1,max_item_A4,max_item_ratio_A4,max_item_B4,max_item_ratio_B4,max_item_C4,max_item_ratio_C4,max_item_D4,max_item_ratio_D4
0,1,우수_우수,감소,의류,0.482420,외식,0.001094,신선식품,0.008964,nan,NaN,의류,0.594464,외식,0.014112,신선식품,0.005490,nan,NaN
1,2,최우수_최우수,증가,일상용품,0.356117,일상용품,0.001254,nan,NaN,nan,NaN,의류,0.383244,nan,NaN,nan,NaN,일상용품,0.001625
2,3,일반_일반,감소,의류,0.093983,nan,NaN,가공식품,0.723789,nan,NaN,nan,NaN,nan,NaN,가공식품,0.517936,nan,NaN


In [2]:
import pandas as pd

pd.read_csv('C:/Users/han71/Documents/카카오톡 받은 파일/독립변수_1004_수정.csv')

,cust_no,level_ch,target_di,max_item_A1,max_item_ratio_A1,item_cnt_A1,max_item_B1,max_item_ratio_B1,item_cnt_B1,max_item_C1,...,max_item_C4,max_item_ratio_C4,item_cnt_C4,max_item_D4,max_item_ratio_D4,item_cnt_D4,channel_cnt1,channel_cnt4,channel_cnt_ch,channel_status
0,1,우수_우수,감소,의류,0.482420,8,외식,0.001094,1,신선식품,...,신선식품,0.005490,3,NaN,NaN,0,3,3,0,불변
1,2,최우수_최우수,증가,일상용품,0.356117,9,일상용품,0.001254,1,NaN,...,NaN,NaN,0,일상용품,0.001625,2,2,2,0,불변
2,3,일반_일반,감소,의류,0.093983,1,NaN,NaN,0,가공식품,...,가공식품,0.517936,4,NaN,NaN,0,2,1,-1,감소
3,4,일반_일반,감소,의류,0.450734,7,가공식품,0.005514,5,신선식품,...,신선식품,0.041449,6,NaN,NaN,0,3,3,0,불변
4,5,일반_일반,감소,가공식품,0.369221,9,신선식품,0.002930,1,NaN,...,NaN,NaN,0,가공식품,0.011199,1,2,2,0,불변
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19142,19356,일반_일반,증가,NaN,NaN,0,NaN,NaN,0,가공식품,...,가공식품,0.520614,5,NaN,NaN,0,1,2,1,증가
19143,19357,일반_일반,증가,의류,0.467645,2,가공식품,0.322458,2,NaN,...,NaN,NaN,0,NaN,NaN,0,2,2,0,불변
19144,19359,일반_일반,증가,NaN,NaN,0,NaN,NaN,0,신선식품,...,가공식품,0.728336,4,NaN,NaN,0,1,1,0,불변
19145,19360,일반_일반,증가,NaN,NaN,0,NaN,NaN,0,NaN,...,NaN,NaN,0,일상용품,0.365720,4,1,2,1,증가
